# Week 3 Assignment - Part 2

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

#### Part 3 - Explore and cluster the neighborhoods in Toronto:

* Load the dataframe from part one

In [48]:
# The code was removed by Watson Studio for sharing.

In [49]:
dfsortedcombo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Rename column

In [50]:
dfsortedcombo.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=True)
dfsortedcombo.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,M1S,Scarborough,Agincourt,43.794200,-79.262029
2,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577
3,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
4,M8W,Etobicoke,"Alderwood, Long Branch",43.602414,-79.543484


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Convert to json

In [51]:
dfsortedcombo.to_json(path_or_buf='geo_toronto.json', orient='table')

In [52]:
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Examine the data

In [53]:
# Data is in the 'data' field 
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]

{'index': 0,
 'Postcode': 'M5H',
 'Borough': 'Downtown Toronto',
 'Neighborhood': 'Adelaide, King, Richmond',
 'Latitude': 43.6505712,
 'Longitude': -79.3845675}

In [54]:
dfsortedcombo.info()
dfsortedcombo.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
Postcode        103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
Latitude        103 non-null float64
Longitude       103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


(103, 5)

In [55]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dfsortedcombo['Borough'].unique()),
        dfsortedcombo.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Get Toronto coordinates

In [56]:
address = 'Toronto, Ca'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {} latitude, {} longitude.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963 latitude, -79.387207 longitude.


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Map of Toronto with neighborhoods

In [57]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfsortedcombo['Latitude'], dfsortedcombo['Longitude'], dfsortedcombo['Borough'], dfsortedcombo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
         [lat, lng],
         radius=5,
         popup=label,
         color='blue',
         fill=True,
         fill_color='#3186cc',
         fill_opacity=0.7,
         parse_html=False).add_to(map_toronto)

map_toronto

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Latitude and longitude of the first Borough

In [58]:
dfsortedcombo.loc[0, 'Borough']

'Downtown Toronto'

In [59]:
borough_latitude = dfsortedcombo.loc[0, 'Latitude'] # borough latitude value
borough_longitude = dfsortedcombo.loc[0, 'Longitude'] # borough longitude value

borough_name = dfsortedcombo.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name,
                                                               borough_latitude, 
                                                               borough_longitude,))

Latitude and longitude values of Downtown Toronto are 43.65057120000001, -79.3845675.


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Connect to Foursquare

In [60]:
# The code was removed by Watson Studio for sharing.

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Send GET request and examine results

In [61]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e52d5400cc1fd196b4f0d95'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 127,
  'suggestedBounds': {'ne': {'lat': 43.65507120450001,
    'lng': -79.37835988143604},
   'sw': {'lat': 43.64607119550001, 'lng': -79.39077511856397}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c062f964a520e5f720e3',
       'name': 'Four Seasons Centre for the Performing Arts',
       'location': {'address': '145 Queen St. W',
        'crossStreet': 'at University Ave.',
        'lat': 43.650592,
        'lng': -79.38

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Get category type

In [62]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Clean josn and put into pandas dataframe

In [63]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
2,Nathan Phillips Square,Plaza,43.652270,-79.383516
3,The Keg Steakhouse + Bar,Steakhouse,43.649937,-79.384196
4,Shangri-La Toronto,Hotel,43.649129,-79.386557


In [64]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Function to repeat same process above

In [65]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Run the above function on each boroguh and create a new dataframe called toronto_venues

In [66]:
toronto_venues = getNearbyVenues(names=dfsortedcombo['Borough'],
                                   latitudes=dfsortedcombo['Latitude'],
                                   longitudes=dfsortedcombo['Longitude']
                                  )

Downtown Toronto
Scarborough
Scarborough
Etobicoke
Etobicoke
North York
North York
North York
Downtown Toronto
Scarborough
Etobicoke
West Toronto
East Toronto
North York
Downtown Toronto
Downtown Toronto
York
Mississauga
Scarborough
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Scarborough
Scarborough
Scarborough
Etobicoke
Downtown Toronto
Central Toronto
Central Toronto
Central Toronto
York
Downtown Toronto
North York
Scarborough
West Toronto
North York
North York
North York
North York
Scarborough
East York
North York
North York
Downtown Toronto
North York
Central Toronto
North York
Scarborough
Downtown Toronto
Downtown Toronto
Downtown Toronto
West Toronto
Scarborough
North York
Etobicoke
Etobicoke
North York
York
Etobicoke
Etobicoke
Scarborough
North York
Central Toronto
East York
West Toronto
Scarborough
Central Toronto
North York
Central Toronto
Etobicoke
North York
West Toronto
North York
Downtown Toronto
Queen's Park
Downtown Toronto
Central Toronto
Scarbor

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Check size of toronto_venues dataframe

In [67]:
print(toronto_venues.shape)
toronto_venues.head()

(2236, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,43.650571,-79.384568,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
1,Downtown Toronto,43.650571,-79.384568,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
2,Downtown Toronto,43.650571,-79.384568,Nathan Phillips Square,43.652270,-79.383516,Plaza
3,Downtown Toronto,43.650571,-79.384568,The Keg Steakhouse + Bar,43.649937,-79.384196,Steakhouse
4,Downtown Toronto,43.650571,-79.384568,Shangri-La Toronto,43.649129,-79.386557,Hotel


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Count of venues by Neighborhood (Borough)

In [68]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,117,117,117,117,117,117
Downtown Toronto,1309,1309,1309,1309,1309,1309
East Toronto,123,123,123,123,123,123
East York,77,77,77,77,77,77
Etobicoke,75,75,75,75,75,75
Mississauga,12,12,12,12,12,12
North York,238,238,238,238,238,238
Scarborough,95,95,95,95,95,95
West Toronto,171,171,171,171,171,171


In [69]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 uniques categories.


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Analyze each neighborhood

In [70]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column

toronto_onehot = toronto_onehot[ ['Neighborhood'] + [ col for col in toronto_onehot.columns if col != 'Neighborhood' ] ]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Downtown Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,

In [71]:
toronto_onehot.shape

(2236, 269)

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Group rows by neighborhood (borough) and by take the mean of the frequency of occurrence of each category

In [121]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017094,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.008547

In [73]:
toronto_grouped.shape

(10, 269)

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* print each neighborhood (borough) along with the top 5 most common venues

In [74]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.07
1  Sandwich Place  0.06
2            Park  0.05
3            Café  0.05
4     Pizza Place  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.05
2           Restaurant  0.03
3               Bakery  0.03
4  Japanese Restaurant  0.02


----East Toronto----
                venue  freq
0    Greek Restaurant  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3             Brewery  0.04
4                Café  0.04


----East York----
          venue  freq
0   Coffee Shop  0.05
1  Burger Joint  0.04
2          Bank  0.04
3   Pizza Place  0.04
4          Park  0.04


----Etobicoke----
                  venue  freq
0           Pizza Place  0.09
1        Sandwich Place  0.07
2           Coffee Shop  0.07
3  Fast Food Restaurant  0.04
4              Pharmacy  0.04


----Mississauga----
                      venue  freq
0               Coffee Shop  0.1

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Dump into Pandas dataframe

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Sandwich Place,Park,Café,Pizza Place,Restaurant,Sushi Restaurant,Dessert Shop,Pub,Clothing Store
1,Downtown Toronto,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
2,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Café,Brewery,Ice Cream Shop,Pizza Place,Bookstore,Pub,Sandwich Place
3,East York,Coffee Shop,Sporting Goods Shop,Bank,Pharmacy,Pizza Place,Park,Burger Joint,Liquor Store,Sushi Restaurant,Breakfast Spot
4,Etobicoke,Pizza Place,Sandwich Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Café,Grocery Store,Gym,Beer Store,Park


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Run K-Means clustering

In [112]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 3, 2, 1, 4, 1, 0], dtype=int32)

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Define the Toronto_data dataframe

In [113]:
toronto_data = dfsortedcombo[dfsortedcombo['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Create dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [114]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
1,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
2,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
3,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant


#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Visualize the resulting clusters

In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Part 3 (continued) - Explore and cluster the neighborhoods in Toronto:

* Visualize the resulting cluster

In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
1,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
2,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
3,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
4,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
5,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
6,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
8,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
9,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Hotel,Japanese Restaurant,Bar,Park,Seafood Restaurant
